In [2]:
from rutermextract import TermExtractor
from flashtext.keyword import KeywordProcessor

In [125]:
def getAspects():
  f = open('aspects.txt', 'r')
  content = f.read()
  f.close()
  return content.split(";")

aspects = getAspects()

Тестовые задания:

In [126]:
TASK_1 = "На клетчатой бумаге с размером клетки 1×1 изображён параллелограмм. Найдите длину его меньшей диагонали."
TASK_1_FULL = "На клетчатой бумаге с размером клетки 1×1 изображён параллелограмм. Найдите длину его меньшей диагонали. Проведем дополнительные построения, как показано на рисунке. Найдём длину диагонали по теореме Пифагора: BD."
TASK_1_ASPECTS = ["Рисунок на клетчатой бумаге", "Параллелограмм", "Теорема Пифагора"]

TASK_2 = "На клавиатуре телефона 10 цифр, от 0 до 9. Какова вероятность того, что случайно нажатая цифра будет чётной?"
TASK_2_FULL = "На клавиатуре телефона 10 цифр, от 0 до 9. Какова вероятность того, что случайно нажатая цифра будет чётной? На клавиатуре телефона 10 цифр, из них 5 четных: 0, 2, 4, 6, 8. Поэтому вероятность того, что случайно будет нажата четная цифра, равна 5 : 10 = 0,5."
TASK_2_ASPECTS = ["Классическая вероятность"]

TASK_3 = "В треугольнике ABC угол C равен 90°,  тангенс A = дробь, числитель — 33, знаменатель — 4 корень из { 33 }, АС = 4. Найдите АВ."
TASK_3_FULL = "В треугольнике ABC угол C равен 90°,  тангенс A = дробь, числитель — 33, знаменатель — 4 корень из { 33 }, АС = 4. Найдите АВ. Имеем: AB = дробь, числитель — AC, знаменатель — косинус A = дробь, числитель — AC, знаменатель — корень из { дробь, числитель — 1 {1 плюс тангенс в степени 2, знаменатель — A }}= дробь, числитель — 4, знаменатель — корень из { дробь, числитель — 1 {1 плюс дробь, числитель — 33, знаменатель — 16 , знаменатель — } }=4 умножить на корень из { дробь, числитель — 49, знаменатель — 16 }=7."
TASK_3_ASPECTS = ["Прямоугольный треугольник", "Тангенс" ]

In [107]:
# Примитивный поиск подстроки в строке
def search_simple(haystack, needle):
    index = -1
    for i in range(len(haystack)-len(needle)+1):
        success = True
        for j in range(len(needle)):
            if needle[j]!=haystack[i+j]:
                success = False
                break
        if success:
            index = i
            break
    return haystack[index: index + len(needle)]

In [108]:
# Поиск подстроки в строке алгоритмом Кнута–Морриса-Пратта
# def search_kmp(haystack, needle):
def search_kmp(text, pattern):
    pattern = list(pattern)

    shifts = [1] * (len(pattern) + 1)
    shift = 1
    for pos in range(len(pattern)):
        while shift <= pos and pattern[pos] != pattern[pos-shift]:
            shift += shifts[pos-shift]
        shifts[pos+1] = shift

    # do the actual search
    startPos = 0
    matchLen = 0
    for c in text:
        while matchLen == len(pattern) or \
              matchLen >= 0 and pattern[matchLen] != c:
            startPos += shifts[matchLen]
            matchLen -= shifts[matchLen]
        matchLen += 1
        if matchLen == len(pattern):
            return text[startPos: startPos + len(pattern)]

In [109]:
# Поиск ключевых слов, используя словарь. Библиотека flashtext
def search_flashtext(haystack, needle):
    keyword_processor = KeywordProcessor()
    for aspect in aspects:
        keyword_processor.add_keyword(aspect)
    keywords_found = keyword_processor.extract_keywords(haystack)
    return keywords_found

In [150]:
# Поиск ключевых слов. Библиотека rutermextract
def search_rutermextract(haystack):
    result = []
    term_extractor = TermExtractor()
    for term in term_extractor(haystack):
        result.append(term.normalized + " " + str(term.count))
    return result

<h2> 1. Результаты работы алгоритмов, используя для анализа только задание.</h2>

<h3> 1.1. Результат алгоритмов библиотеки FlashText</h3>

In [127]:
for task in [TASK_1, TASK_2, TASK_3]:
  result = search_flashtext(task, aspects)
  print("Result for task is ", result)

Result for task is  ['Параллелограмм']
Result for task is  ['Вероятность']
Result for task is  ['Тангенс']


<h3> 1.2. Результаты алгоритмов библиотеки Rutermextract</h3>

In [151]:
for task in [TASK_1, TASK_2, TASK_3]:
  result = search_rutermextract(task)
  print("Result for task is ", result)

Result for task is  ['размер клетки 1', 'малейшая диагональ 1', 'клетчатая бумага 1', 'параллелограмм 1', 'длина 1']
Result for task is  ['нажатая цифра 1', 'клавиатура телефона 1', '10 цифр 1', 'вероятность 1']
Result for task is  ['угол c 1', 'треугольник abc 1', 'тангенс a 1', '4 корень 1', 'числитель 1', 'знаменатель 1', 'дробь 1', 'ас 1', 'авы 1']


<h3> 1.3. Результат примитивного поиска</h3>

In [129]:
for task in [TASK_1, TASK_2, TASK_3]:
  result = []
  for aspect in aspects:
    res_iter = search_simple(task, aspect)
    if (res_iter):
      result.append(res_iter)
  print("Result for task is ", result)

Result for task is  []
Result for task is  []
Result for task is  []


<h3> 1.4. Результат алгоритма Кнута-Морриса-Прата</h3>

In [130]:
for task in [TASK_1, TASK_2, TASK_3]:
  result = []
  for aspect in aspects:
    res_iter = search_kmp(task, aspect)
    if (res_iter):
      result.append(res_iter)
  print("Result for task is ", result)

Result for task is  []
Result for task is  []
Result for task is  []


*Используем предобработку данных для работы алгоритмов по поиску подстроки в строке*

In [131]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alena.ogloblina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [132]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

In [133]:
def preprocess_text(text):
  tokens = mystem.lemmatize(text.lower())
  tokens = [token for token in tokens if token not in russian_stopwords\
            and token != " " \
            and token.strip() not in punctuation]
  
  text = " ".join(tokens)
  return text

In [134]:
TASK_LEMMS = [preprocess_text(TASK_1), preprocess_text(TASK_2), preprocess_text(TASK_3)]
TASK_FULL_LEMMS = [preprocess_text(TASK_1_FULL), preprocess_text(TASK_2_FULL), preprocess_text(TASK_3_FULL)]

In [135]:
lem_aspect = []

for aspect in aspects:
  res = preprocess_text(aspect)
  lem_aspect.append(res)

Алгоритм примитивного поиска после лемматизации задания и аспектов:

In [145]:
for task in TASK_LEMMS:
  result = []
  for aspect in lem_aspect:
    res_iter = search_simple(task, aspect)
    if (res_iter):
      result.append(res_iter)
  print("Result for task is ", result)

Result for task is  ['параллелограмм']
Result for task is  ['вероятность']
Result for task is  ['тангенс']


Алгоритм поиска Кнутта-Морриса-Прата после лемматизации задания и аспектов:

In [142]:
for task in TASK_LEMMS:
  result = []
  for aspect in lem_aspect:
    res_iter = search_kmp(task, aspect)
    if (res_iter):
      result.append(res_iter)
  print("Result for task is ", result)

Result for task is  ['параллелограмм']
Result for task is  ['вероятность']
Result for task is  ['тангенс']


<h2> 2. Результаты работы алгоритмов, используя для анализа задание и решение.</h2>

In [153]:
for task in [TASK_1_FULL, TASK_2_FULL, TASK_3_FULL]:
  result = search_flashtext(task, aspects)
  print("Result for task is ", result)

Result for task is  ['Параллелограмм']
Result for task is  ['Вероятность', 'Вероятность']
Result for task is  ['Тангенс', 'Синус', 'Тангенс']


In [154]:
for task in [TASK_1_FULL, TASK_2_FULL, TASK_3_FULL]:
  result = search_rutermextract(task)
  print("Result for task is ", result)

Result for task is  ['длина 2', 'теорема пифагора 1', 'размер клетки 1', 'малейшая диагональ 1', 'клетчатая бумага 1', 'дополнительные построение 1', 'рисунок 1', 'параллелограмм 1', 'диагональ 1', 'bd 1']
Result for task is  ['клавиатура телефона 2', '10 цифр 2', 'вероятность 2', 'чётная цифра 1', 'нажатая цифра 1']
Result for task is  ['числитель 8', 'знаменатель 8', 'дробь 8', 'корень 3', 'ac 2', 'угол c 1', 'треугольник abc 1', 'тангенс a 1', 'косинус a 1', '4 корень 1', 'тангенс 1', 'степень 1', 'ас 1', 'авы 1', 'ab 1', 'a 1']


In [155]:
for task in TASK_FULL_LEMMS:
  result = []
  for aspect in lem_aspect:
    res_iter = search_simple(task, aspect)
    if (res_iter):
      result.append(res_iter)
  print("Result for task is ", result)

Result for task is  ['теорема пифагор', 'параллелограмм']
Result for task is  ['вероятность']
Result for task is  ['синус', 'тангенс']


In [156]:
for task in TASK_FULL_LEMMS:
  result = []
  for aspect in lem_aspect:
    res_iter = search_kmp(task, aspect)
    if (res_iter):
      result.append(res_iter)
  print("Result for task is ", result)

Result for task is  ['теорема пифагор', 'параллелограмм']
Result for task is  ['вероятность']
Result for task is  ['синус', 'тангенс']
